In [ ]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = 5) #input = 28*28*1, output = 24*24*8
        self.bn1 = nn.BatchNorm2d(8)
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = 5) # input = 24*24*8, output = 20*20*16 
        self.bn2 = nn.BatchNorm2d(16) 
        # -> maxpool -> 10*10*16
        self.conv3 = nn.Conv2d(in_channels = 16, out_channels = 32, kernel_size = 5) # input = 10*10*16, output = 6*6*32
        self.bn3 = nn.BatchNorm2d(32)
        self.dropout1 = nn.Dropout2d(0.1)
        self.conv4 = nn.Conv2d(in_channels = 32, out_channels = 24, kernel_size = 1, stride = 1) # input = 6*6*32, output = 6*6*24
        self.gap = nn.AvgPool2d(kernel_size = 6) # kernel_size = 6*6 to get class specific feature
        self.fc_output = nn.Linear(in_features = 24, out_features = 10) # input = 24, output = 10
      
    def forward(self, tensor):
        x = tensor 
        x = self.bn1(F.relu(self.conv1(x)))
        x = self.bn2(F.relu(self.conv2(x)))
        x = F.max_pool2d(x, kernel_size = 2, stride = 2) 
        x = self.dropout1(self.bn3(F.relu(self.conv3(x))))
        x = F.relu(self.conv4(x))
        x = self.gap(x)
        x = x.reshape(x.shape[0], -1)
        x = self.fc_output(x)
        return F.log_softmax(x, dim = 1)

In [ ]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 24, 24]             208
       BatchNorm2d-2            [-1, 8, 24, 24]              16
            Conv2d-3           [-1, 16, 20, 20]           3,216
       BatchNorm2d-4           [-1, 16, 20, 20]              32
            Conv2d-5             [-1, 32, 6, 6]          12,832
       BatchNorm2d-6             [-1, 32, 6, 6]              64
         Dropout2d-7             [-1, 32, 6, 6]               0
            Conv2d-8             [-1, 24, 6, 6]             792
         AvgPool2d-9             [-1, 24, 1, 1]               0
           Linear-10                   [-1, 10]             250
Total params: 17,410
Trainable params: 17,410
Non-trainable params: 0
--------------------------------------------------------------

In [ ]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

In [ ]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [ ]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 16):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

loss=0.0471455343067646 batch_id=468: 100%|██████████| 469/469 [00:14<00:00, 33.04it/s]



Test set: Average loss: 0.0923, Accuracy: 9758/10000 (98%)



loss=0.057915911078453064 batch_id=468: 100%|██████████| 469/469 [00:14<00:00, 32.39it/s]



Test set: Average loss: 0.0625, Accuracy: 9816/10000 (98%)



loss=0.023455316200852394 batch_id=468: 100%|██████████| 469/469 [00:14<00:00, 32.56it/s]


...


Test set: Average loss: 0.0246, Accuracy: 9926/10000 (99%)



loss=0.08837571740150452 batch_id=468: 100%|██████████| 469/469 [00:14<00:00, 32.99it/s]



Test set: Average loss: 0.0216, Accuracy: 9933/10000 (99%)



loss=0.09227914363145828 batch_id=468: 100%|██████████| 469/469 [00:14<00:00, 32.77it/s]



Test set: Average loss: 0.0253, Accuracy: 9927/10000 (99%)

